In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd


def load_files_from_dir(path):
    data = {}
    list_files = list(path.glob('*.npz'))
    for p in tqdm(list_files):
        try:
            d = np.load(p)
        except:
            continue
        for k, v in d.items():
            data.setdefault(k, []).extend( v)
    if not data:
        try:
            data = np.load(path.with_suffix('.npz'))
        except:
            raise FileNotFoundError(f'No files found in {path}')
    indices = np.argsort(data['time_ns'])
    data = {k:np.stack(v)[indices] for k, v in data.items()}
    return data


path_to_files = Path('/mnt/e/Downloads/meas/B/')

Load data

In [ ]:
pan = load_files_from_dir(path_to_files / 'pan')
mono = load_files_from_dir(path_to_files / 'mono')
list(mono.keys())

In [ ]:
idx = np.searchsorted(pan['time_ns'], mono['time_ns'].max())
# pan = {k:v[:idx] for k, v in pan.items()}
# np.savez(path_to_files / 'pan_.npz', **pan)

idx = np.searchsorted(mono['time_ns'], pan['time_ns'].min())
print(f'Number of frames in pan data: {len(pan["time_ns"])}')
print(f'Number of frames in mono data: {len(mono["time_ns"])}')
print(f'The idx in pan with the highest mono time: {idx}')
# mono = {k:v[idx:] for k, v in mono.items()}
# np.savez(path_to_files / 'mono_.npz', **mono)



Find the closest neighbors between pan and mono (in terms of time_ns)

In [ ]:
# # Find the first common index
# if len(pan['time_ns']) < len(mono['time_ns']):
#     idx = np.searchsorted(mono['time_ns'], pan['time_ns'][0])
#     for k in mono.keys():
#         mono[k] = mono[k][idx:]
# else:
#     idx = np.searchsorted(pan['time_ns'], mono['time_ns'][0])
#     for k in pan.keys():
#         pan[k] = pan[k][idx:]

indices = np.array(sorted(set(np.concatenate([mono['time_ns'] , pan['time_ns']]))))
indices_mono = np.searchsorted(indices, mono['time_ns'])
indices_pan = np.searchsorted(indices, pan['time_ns'])


plt.figure()
plt.scatter( indices_pan, pan['time_ns'], label='pan', marker='x', s=0.5)
plt.scatter( indices_mono,  mono['time_ns'], label='mono', marker='o', s=0.5)
plt.grid()
plt.legend()
plt.ylabel('Time [s]')
plt.show()


In [ ]:
diff = np.abs(pan['time_ns'][:, None] - mono['time_ns'][None, :])
# diff = np.argmin(diff, axis=0)
# diff
# plt.plot(diff)





# diff = np.min(diff, axis=0)
# diff = diff / 1e9
# # diff = [(pan['time_ns'][p] - q) / 1e9 for p, q in zip(diff, mono['time_ns'])]
# plt.plot(diff[diff < 0.1])

# # [p for p in (diff / 1e9)]
# # 
# # for i in range(len(pan['time_ns'])):
# #     dict_of_connections.setdefault(pan['time_ns'][i], []).append(mono['time_ns'][i])

Build a dict with all the data sorted by the time_ns indices

In [ ]:
rate_mono = len(mono['time_ns']) / ((mono['time_ns'][-1] - mono['time_ns'][0]) / 1e9)
rate_pan = len(pan['time_ns']) / ((pan['time_ns'][-1] - pan['time_ns'][0]) / 1e9)
print(f'rate_mono: {rate_mono}')
print(f'rate_pan: {rate_pan}')

In [ ]:
# # plot the first frame in each file side by side
# plt.figure()
# plt.subplot(121)
# plt.imshow(mono['frames'][0])
# plt.title('Mono')
# plt.subplot(122)
# plt.imshow(pan['frames'][0])
# plt.title('Pan')
# plt.show()

# Plot the first frame in each file on top of each other
CROP_H , CROP_W = 100, 120
plt.figure(figsize=(20, 12))
plt.imshow(np.concatenate([pan['frames'][0][CROP_H:-CROP_H, CROP_W:-CROP_W], 
                           mono['frames'][0][CROP_H:-CROP_H, CROP_W:-CROP_W]], axis=1))
plt.show()



Plot the FPAs and the update times of the FPAs

In [ ]:
indices_pan = (pan['time_ns'] - pan['time_ns'][0]) / 1e9
mark_update_pan_times = np.diff(pan['fpa_update_time'], prepend=pan['fpa_update_time'][0]) != 0
indices_marks_pan = indices_pan[mark_update_pan_times]
mark_update_pan_times = pan['fpa'][mark_update_pan_times]
indices_mono = (mono['time_ns'] - mono['time_ns'][0]) / 1e9
mark_update_mono_times = np.diff(mono['fpa_update_time'], prepend=mono['fpa_update_time'][0]) != 0
indices_marks_mono = indices_mono[mark_update_mono_times]
mark_update_mono_times = mono['fpa'][mark_update_mono_times]

plt.figure()
plt.plot(indices_pan, pan['fpa'], label='pan', c='r')
plt.plot(indices_mono, mono['fpa'], label='mono', c='b')
plt.scatter(indices_marks_pan, mark_update_pan_times, c='r', label='pan updates')
plt.scatter(indices_marks_mono, mark_update_mono_times, c='b', label='mono updates')
plt.legend()
plt.grid()
plt.show()


To Dataframe

In [ ]:
images = np.split(pan['frames'], indices_or_sections=len(pan['frames']))
pd.DataFrame(data=[p.squeeze() for p in images], index=pan['time_ns']).head()
# df_pan = pd.DataFrame(pan['fpa'], index=pan['time_ns']).sort_index()
# df_mono = pd.DataFrame(mono['fpa'], index=mono['time_ns']).sort_index()

In [ ]:
df_mono.head()